In [ ]:
# notebook settings
%load_ext autoreload
%autoreload 2
# %matplotlib notebook

# external imports
import numpy as np
import sympy as sp
from scipy.linalg import solve_discrete_are
import matplotlib.pyplot as plt

# internal imports
from dynamics import mld
from warm_start_hmpc.controller import HybridModelPredictiveController

In [ ]:
# controller horizon
T = 40

# weight matrices
C = np.eye(mld.nx) * .1
# C_T = np.eye(mld.nx) * 1.
D = np.vstack([1.]+[0.]*(mld.nu - 1)).T * .1

# terminal cost
Bu = mld.B[:,0:1]
Du = D[:,0:1]
P = solve_discrete_are(mld.A, Bu, C.T.dot(C), Du.T.dot(Du))
C_T = np.linalg.cholesky(P).T
objective = [C, D, C_T]

# terminal constraints
F_T = np.vstack((np.eye(mld.nx), - np.eye(mld.nx)))
h_T = np.ones(2*mld.nx)*.1
terminal_set = [F_T, h_T]

# hybrid controller
controller = HybridModelPredictiveController(mld, T, objective, terminal_set)

In [ ]:
x0 = np.array([0., 0., 1., 0.])

In [ ]:
# solve in open loop
solution, leaves = controller.feedforward(
    x0,
    draw_label='Cart pole with walls'
)

In [ ]:
e0 = np.random.randn(mld.nx)*0.
uc0 = solution.variables['uc'][0]
ub0 = solution.variables['ub'][0]
warm_start = controller.construct_warm_start(leaves, x0, uc0, ub0, e0)

In [ ]:
u0 = np.concatenate((uc0, ub0))
x1 = mld.A.dot(x0) + mld.B.dot(u0) + e0
solution, leaves = controller.feedforward(
    x1,
    warm_start=warm_start,
    draw_label='Cart pole with walls warm start'
)

In [ ]:
solution, leaves = controller.feedforward(x1)

In [ ]:
from visualizer import vis, animate
vis.jupyter_cell()

In [ ]:
animate(solution.variables['x'])